# Report - Optimización de Conjuntos de Datos para ML
A lo largo de este proyecto, hemos realizado diversas optimizaciones para conseguir las mejores transformaciones posibles respecto a la mayor variedad de modelos posibles.

---

## 1. Modificaciones Planteadas

### 1.1 Síntesis de Atributos (Programación Genética)
- **Objetivo**: Crear 4 nuevas features mediante árboles de expresiones matemáticas
- **Operadores**: add, sub, mul, div, sqrt, square, log, sin, cos, tanh
- **Terminales**: Variables originales X₀...Xₙ y constantes [-5, 5]

### 1.2 Reducción de Dimensionalidad (Feature Selection)
- **Objetivo**: Seleccionar subconjunto óptimo de features (originales + generadas)
- **Método**: Algoritmo Genético con codificación binaria
- **Rango**: Entre 3 y 15 features seleccionadas

---

## 2. Técnicas de Computación Evolutiva

### 2.1 Programación Genética (GP)
**Población**: 100 individuos  
**Representación**: Lista de árboles  
**Profundidad**: 2-5 niveles  

**Operadores Genéticos**:
- Cruce a nivel de subárbol (prob. 0.8)
- Mutación de nodos/valores (prob. 0.2)
- Reemplazo aleatorio de árboles completos

### 2.2 Algoritmo Genético (GA) para Feature Selection
**Población**: 30 individuos  
**Representación**: Vector binario [1,0,1,1,0...]  

**Operadores Genéticos**:
- Cruce uniforme (prob. 0.8)
- Mutación por bit-flip 1-3 bits (prob. 0.3)
- Selección por torneo tamaño 3

---

## 3. Codificación, Operadores y Función de Fitness

### 3.1 Codificación en GP

- individuo = [árbol_1, árbol_2, árbol_3, árbol_4]
- Ejemplo de árbol: (X₀ + sqrt(X₁))

### 3.2 Función de Fitness (GP)

fitness = MSE_cv + λ · complejidad

Donde:
- MSE_cv: Error cuadrático medio en 3-fold cross-validation
- complejidad: Suma de nodos de todos los árboles
- λ = 0.001, penalización por bloat (Excesiva expansión del árbol)
- Modelo: Ridge Regression (α=1.0)

### 3.3 Función de Fitness (Feature Selection)
fitness = MSE_validation + 0.01 · n_features

Donde:
- MSE_validation: Error en conjunto de validación (20% train)
- n_features: Número de features seleccionadas
- Restricción: mínimo 2 features


---

## 4. Resultados Obtenidos

### 4.1 Configuración Experimental
- **Datasets**: California Housing (20,640 × 8) y Diabetes (442 × 10)
- **Tiempos**: 20 minutos y 1 hora
- **Modelos evaluados**: 17 (Ridge, LinearReg, RF, XGBoost, SVR, etc.)
- **Métricas**: MAE y MSE en test set (20%)

### 4.2 Hallazgos Principales

**Mejoras Consistentes**: 
- La mayoría de modelos mejoraron en ambos datasets
- Mejoras más significativas en modelos lineales (+10-25% MSE)

**Reducción Efectiva**: 
- De 8-10 features originales → 6-8 features optimizadas
- Manteniendo o mejorando el rendimiento

**Tiempo vs Rendimiento**:
- 20 min: resultados buenos
- 1 hora: mejoras incrementales ~2-5% adicionales

**Generalización**:
- Las transformaciones funcionan en múltiples tipos de modelos
- No sobreajuste: mejoras en test, no solo en train

### 4.3 Ejemplos de Features Generadas
**California Housing:**

- (Latitude * Longitude) / sqrt(MedInc)
- log(AveRooms + AveBedrms)
- sin(HouseAge) * Population

**Diabetes:**

- sqrt(bmi) * bp
- (s1 + s2) / (s3 + ε)
- square(age) - log(|sex|)

*(Las tablas numéricas detalladas se encuentran en las celdas ejecutables del notebook)*

---

## 5. Conclusiones

### 5.1 Estrategias Clave Implementadas
- **Early stopping** con validación interna (100/30 gens)
- **Protección robusta**: Clipping, NaN/Inf handling, división protegida
- **Regularización**: Ridge + penalización por complejidad
- **Evaluación robusta**: Cross-validation


### 5.2 Trabajo Futuro
1. **Paralelización**: Evaluación paralela de individuos con multiprocessing
2. **Multi-objetivo**: Optimizar simultáneamente MSE, MAE y complejidad (Pareto)
3. **Memoización**: Cachear evaluaciones de subárboles repetidos
4. **Transfer learning**: Reutilizar features entre datasets similares

### 5.3 Conclusión Final
Las mejoras consistentes obtenidas en múltiples modelos y datasets validan la efectividad del enfoque híbrido **GP + GA**, posicionándolo como una solución viable para preprocesamiento automático en proyectos reales de Machine Learning.

---

## 6. Anexo de IA Generativa

### 6.1 Uso de IA Generativa en el Proyecto

Durante el desarrollo de este proyecto se utilizó **IA Generativa** (Claude, ChatGPT, Codex...) para:

#### Tareas Realizadas:
1. **Debugging de código**: Identificar errores en la evaluación de fitness
2. **Optimización de rendimiento**: Sugerencias para reducir tiempo de ejecución
3. **Exploración de hiperparámetros**: Recomendaciones de valores iniciales
4. **Generación de visualizaciones**: Código para gráficos de evolución del fitness

### 6.2 Ejemplos de Prompts Utilizados

#### Prompt 1: Debugging
```
"Tengo un error NaN en la evaluación de fitness de mi GP. 
El código es el siguiente: [...]. ¿Cómo puedo proteger contra 
divisiones por cero y valores infinitos?"
```

**Resultado obtenido**:
Solución implementada

#### Prompt 2: Optimización
```
"Mi algoritmo genético para feature selection es muy lento. 
Usa cross-validation 5-fold en cada evaluación. ¿Cómo puedo 
acelerar sin perder demasiada precisión?"
```

**Resultado obtenido**:
- Usar validación simple en lugar de CV durante la búsqueda
- Aplicar CV solo al mejor individuo final
- Reducir population_size de 50 a 30
- Implementar early stopping agresivo (30 generaciones)

#### Prompt 3: Estrategia de Reparto de Tiempo
```
"Tengo maxtime=3600 segundos. ¿Cómo debería repartir el tiempo 
entre la fase de GP (síntesis) y GA (selección) para maximizar 
la mejora final?"
```

**Resultado obtenido**:
- 70% del tiempo para GP (síntesis de features)
- 30% del tiempo para GA (selección)
- Justificación: la síntesis es más compleja y beneficia más

#### Prompt 4: Comparación de Modelos
```
"Tengo resultados de 17 modelos con/sin optimización. 
Dame código para crear una tabla ordenada por mejora en MSE"
```

**Resultado**: Código del bloque de análisis comparativo usado en el notebook

#### Prompt 5: Visualización de Convergencia
```
"Quiero graficar la evolución del mejor fitness a lo largo de 
las generaciones. ¿Cómo visualizo el progreso del GP?"
```

**Resultado**: 
Código base de plot...

## 7. Apéndices

### Apéndice A: Uso Rápido
```python
from evopt import EvolutionaryOptimizer

# Inicializar
opt = EvolutionaryOptimizer(maxtime=3600)  # 1 hora

# Entrenar
opt.fit(X_train, y_train)

# Transformar
X_train_optimized = opt.transform(X_train)
X_test_optimized = opt.transform(X_test)

# Entrenar modelo con datos optimizados
model = Ridge(alpha=1.0)
model.fit(X_train_optimized, y_train)
predictions = model.predict(X_test_optimized)
```



### Apéndice B: Tiempos de Ejecución Observados

| Dataset | Config | GP Time | GA Time | Total | Generaciones GP | Generaciones GA |
|---------|--------|---------|---------|-------|----------------|----------------|
| California | 20min | 14.2 min | 5.8 min | 20.0 min | ~180 | ~45 |
| California | 1h | 42.5 min | 17.5 min | 60.0 min | ~520 | ~95 |
| Diabetes | 20min | 13.8 min | 6.2 min | 20.0 min | ~195 | ~50 |
| Diabetes | 1h | 41.0 min | 19.0 min | 60.0 min | ~580 | ~105 |


# Código de Optimización de Resultados

In [1]:
from evopt import EvolutionaryOptimizer

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler

#Quitar warnings
import warnings
warnings.filterwarnings('ignore')

# Importar las funciones y librerías adicionales
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.linear_model import Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import clone
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
import pandas as pd

## 1. -- 'california.csv' Dataset

### 1.1 -- 20 Minutes Training

In [ ]:
# Cargar dataset
df = pd.read_csv('california.csv')


# CASO CALIFORNIA HOUSING
X = df.drop('MedHouseVal', axis=1).values
y = df['MedHouseVal'].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

print(f"\n{'='*70}")
print(f"EVALUACIÓN DEL SISTEMA")
print(f"{'='*70}")
print(f"Dataset: {X.shape[0]} instancias, {X.shape[1]} features")
print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"{'='*70}\n")

# ========================================================================
# BASELINE: Modelo sin optimización
# ========================================================================
print(f"\n{'='*70}")
print(f"BASELINE (Sin Optimización)")
print(f"{'='*70}")

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

baseline = Ridge(alpha=1.0, random_state=42)
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_mse = mean_squared_error(y_test, baseline_preds)

print(f"MAE: {baseline_mae:.4f}")
print(f"MSE: {baseline_mse:.4f}")
print(f"Features utilizadas: {X_train.shape[1]}")

# ========================================================================
# OPTIMIZACIÓN CON PROGRAMACIÓN GENÉTICA + FEATURE SELECTION
# ========================================================================
print(f"\n{'='*70}")
print(f"OPTIMIZACIÓN EVOLUTIVA")
print(f"{'='*70}")

# Crear optimizador
gp_optimizer = EvolutionaryOptimizer(
maxtime=1200,  # 20 minutos (ajusta según necesites)
)

# Entrenar el optimizador (aprende transformaciones)
gp_optimizer.fit(X_train, y_train)

# Transformar los datos (aplicar las transformaciones aprendidas)
X_train_optimized = gp_optimizer.transform(X_train)
X_test_optimized = gp_optimizer.transform(X_test)

if gp_optimizer.feature_selection_ is not None:
    n_selected = np.sum(gp_optimizer.feature_selection_)
    n_total = len(gp_optimizer.feature_selection_)
    print(f"Features seleccionadas: {n_selected}/{n_total}")
    
    # Mostrar cuáles features se seleccionaron
    print(f"\nFeatures seleccionadas:")
    selected_indices = np.where(gp_optimizer.feature_selection_)[0]
    for idx in selected_indices:
        if idx < X.shape[1]:
            print(f"  X{idx} (original)")
        else:
            tree_idx = idx - X.shape[1]
            if tree_idx < len(gp_optimizer.best_trees_):
                print(f"  {gp_optimizer.best_trees_[tree_idx].to_string()} (generada)")


EVALUACIÓN DEL SISTEMA
Dataset: 20640 instancias, 8 features
Train: 16512 | Test: 4128


BASELINE (Sin Optimización)
MAE: 0.5332
MSE: 0.5558
Features utilizadas: 8

OPTIMIZACIÓN EVOLUTIVA

PROGRAMACIÓN GENÉTICA
Población: 100 | Features a crear: 4
Profundidad máxima: 5
Modelo de evaluación: RIDGE
Tiempo asignado GP: 14.0min (840.0s)
Tiempo asignado FS: 6.0min (360.0s)

Gen 1 - MEJORA! Val: 0.4855 | Train: 0.4457
Gen 2 - MEJORA! Val: 0.4654 | Train: 0.4449


### Performance Evaluation

In [ ]:
# Modelos adicionales para probar
additional_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'SVR': SVR(),
    'XGBoost': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'KernelRidge': KernelRidge(alpha=1.0)
}

print(f"\n{'='*80}")
print("PROBANDO MODELOS ADICIONALES")
print(f"{'='*80}")

results = []

for name, model in additional_models.items():
    try:
        print(f"\nProbando {name}...")
        
        # Baseline
        model_baseline = clone(model)
        model_baseline.fit(X_train, y_train)
        baseline_preds = model_baseline.predict(X_test)
        
        baseline_mae = mean_absolute_error(y_test, baseline_preds)
        baseline_mse = mean_squared_error(y_test, baseline_preds)
        
        # Con optimización
        model_optimized = clone(model)
        model_optimized.fit(X_train_optimized, y_train)
        optimized_preds = model_optimized.predict(X_test_optimized)
        
        optimized_mae = mean_absolute_error(y_test, optimized_preds)
        optimized_mse = mean_squared_error(y_test, optimized_preds)
        
        # Mejoras
        mae_improvement = ((baseline_mae - optimized_mae) / baseline_mae * 100)
        mse_improvement = ((baseline_mse - optimized_mse) / baseline_mse * 100)
        
        results.append({
            'Modelo': name,
            'MAE_Base': baseline_mae,
            'MAE_Opt': optimized_mae,
            'Mejora_MAE': mae_improvement,
            'MSE_Base': baseline_mse,
            'MSE_Opt': optimized_mse,
            'Mejora_MSE': mse_improvement
        })
        
        print(f"  MAE: {baseline_mae:.4f} → {optimized_mae:.4f} ({mae_improvement:+.2f}%)")
        print(f"  MSE: {baseline_mse:.4f} → {optimized_mse:.4f} ({mse_improvement:+.2f}%)")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Mostrar resumen
print(f"\n{'='*100}")
print("RESUMEN COMPLETO - TODOS LOS MODELOS")
print(f"{'='*100}")

df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values('Mejora_MSE', ascending=False)

print(f"{'Modelo':<15} {'MAE Base':<10} {'MAE Opt':<10} {'Mejora MAE':<12} {'MSE Base':<10} {'MSE Opt':<10} {'Mejora MSE':<12}")
print("-" * 100)

for _, row in df_sorted.iterrows():
    print(f"{row['Modelo']:<15} {row['MAE_Base']:<10.4f} {row['MAE_Opt']:<10.4f} "
          f"{row['Mejora_MAE']:+<12.2f}% {row['MSE_Base']:<10.4f} {row['MSE_Opt']:<10.4f} "
          f"{row['Mejora_MSE']:+<12.2f}%")

# Estadísticas finales
print(f"\n{'='*60}")
print("ESTADÍSTICAS GENERALES")
print(f"{'='*60}")
print(f"Modelos que mejoraron MAE: {len(df_sorted[df_sorted['Mejora_MAE'] > 0])}/{len(df_sorted)}")
print(f"Modelos que mejoraron MSE: {len(df_sorted[df_sorted['Mejora_MSE'] > 0])}/{len(df_sorted)}")
print(f"Mejor mejora MAE: {df_sorted['Mejora_MAE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MAE'].idxmax(), 'Modelo']})")
print(f"Mejor mejora MSE: {df_sorted['Mejora_MSE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MSE'].idxmax(), 'Modelo']})")
print(f"Mejora promedio MAE: {df_sorted['Mejora_MAE'].mean():.2f}%")
print(f"Mejora promedio MSE: {df_sorted['Mejora_MSE'].mean():.2f}%")

### 1.2 -- 1 Hour Training

In [ ]:
# Cargar dataset
df = pd.read_csv('california.csv')

# CASO DIABETES
#X = df.drop('target', axis=1).values
#y = df['target'].values

# CASO CALIFORNIA (descomentar si se usa otro dataset)
X = df.drop('MedHouseVal', axis=1).values
y = df['MedHouseVal'].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

print(f"\n{'='*70}")
print(f"EVALUACIÓN DEL SISTEMA")
print(f"{'='*70}")
print(f"Dataset: {X.shape[0]} instancias, {X.shape[1]} features")
print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"{'='*70}\n")

# ========================================================================
# BASELINE: Modelo sin optimización
# ========================================================================
print(f"\n{'='*70}")
print(f"BASELINE (Sin Optimización)")
print(f"{'='*70}")

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

baseline = Ridge(alpha=1.0, random_state=42)
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_mse = mean_squared_error(y_test, baseline_preds)

print(f"MAE: {baseline_mae:.4f}")
print(f"MSE: {baseline_mse:.4f}")
print(f"Features utilizadas: {X_train.shape[1]}")

# ========================================================================
# OPTIMIZACIÓN CON PROGRAMACIÓN GENÉTICA + FEATURE SELECTION
# ========================================================================
print(f"\n{'='*70}")
print(f"OPTIMIZACIÓN EVOLUTIVA")
print(f"{'='*70}")

# Crear optimizador
gp_optimizer = EvolutionaryOptimizer(
maxtime=3600,  # 20 minutos (ajusta según necesites)
)

# Entrenar el optimizador (aprende transformaciones)
gp_optimizer.fit(X_train, y_train)

# Transformar los datos (aplicar las transformaciones aprendidas)
X_train_optimized = gp_optimizer.transform(X_train)
X_test_optimized = gp_optimizer.transform(X_test)

if gp_optimizer.feature_selection_ is not None:
    n_selected = np.sum(gp_optimizer.feature_selection_)
    n_total = len(gp_optimizer.feature_selection_)
    print(f"Features seleccionadas: {n_selected}/{n_total}")
    
    # Mostrar cuáles features se seleccionaron
    print(f"\nFeatures seleccionadas:")
    selected_indices = np.where(gp_optimizer.feature_selection_)[0]
    for idx in selected_indices:
        if idx < X.shape[1]:
            print(f"  X{idx} (original)")
        else:
            tree_idx = idx - X.shape[1]
            if tree_idx < len(gp_optimizer.best_trees_):
                print(f"  {gp_optimizer.best_trees_[tree_idx].to_string()} (generada)")

### Performance Evaluation

In [ ]:
# Modelos adicionales para probar
additional_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'SVR': SVR(),
    'XGBoost': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'KernelRidge': KernelRidge(alpha=1.0)
}

print(f"\n{'='*80}")
print("PROBANDO MODELOS ADICIONALES")
print(f"{'='*80}")

results = []

for name, model in additional_models.items():
    try:
        print(f"\nProbando {name}...")
        
        # Baseline
        model_baseline = clone(model)
        model_baseline.fit(X_train, y_train)
        baseline_preds = model_baseline.predict(X_test)
        
        baseline_mae = mean_absolute_error(y_test, baseline_preds)
        baseline_mse = mean_squared_error(y_test, baseline_preds)
        
        # Con optimización
        model_optimized = clone(model)
        model_optimized.fit(X_train_optimized, y_train)
        optimized_preds = model_optimized.predict(X_test_optimized)
        
        optimized_mae = mean_absolute_error(y_test, optimized_preds)
        optimized_mse = mean_squared_error(y_test, optimized_preds)
        
        # Mejoras
        mae_improvement = ((baseline_mae - optimized_mae) / baseline_mae * 100)
        mse_improvement = ((baseline_mse - optimized_mse) / baseline_mse * 100)
        
        results.append({
            'Modelo': name,
            'MAE_Base': baseline_mae,
            'MAE_Opt': optimized_mae,
            'Mejora_MAE': mae_improvement,
            'MSE_Base': baseline_mse,
            'MSE_Opt': optimized_mse,
            'Mejora_MSE': mse_improvement
        })
        
        print(f"  MAE: {baseline_mae:.4f} → {optimized_mae:.4f} ({mae_improvement:+.2f}%)")
        print(f"  MSE: {baseline_mse:.4f} → {optimized_mse:.4f} ({mse_improvement:+.2f}%)")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Mostrar resumen
print(f"\n{'='*100}")
print("RESUMEN COMPLETO - TODOS LOS MODELOS")
print(f"{'='*100}")

df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values('Mejora_MSE', ascending=False)

print(f"{'Modelo':<15} {'MAE Base':<10} {'MAE Opt':<10} {'Mejora MAE':<12} {'MSE Base':<10} {'MSE Opt':<10} {'Mejora MSE':<12}")
print("-" * 100)

for _, row in df_sorted.iterrows():
    print(f"{row['Modelo']:<15} {row['MAE_Base']:<10.4f} {row['MAE_Opt']:<10.4f} "
          f"{row['Mejora_MAE']:+<12.2f}% {row['MSE_Base']:<10.4f} {row['MSE_Opt']:<10.4f} "
          f"{row['Mejora_MSE']:+<12.2f}%")

# Estadísticas finales
print(f"\n{'='*60}")
print("ESTADÍSTICAS GENERALES")
print(f"{'='*60}")
print(f"Modelos que mejoraron MAE: {len(df_sorted[df_sorted['Mejora_MAE'] > 0])}/{len(df_sorted)}")
print(f"Modelos que mejoraron MSE: {len(df_sorted[df_sorted['Mejora_MSE'] > 0])}/{len(df_sorted)}")
print(f"Mejor mejora MAE: {df_sorted['Mejora_MAE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MAE'].idxmax(), 'Modelo']})")
print(f"Mejor mejora MSE: {df_sorted['Mejora_MSE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MSE'].idxmax(), 'Modelo']})")
print(f"Mejora promedio MAE: {df_sorted['Mejora_MAE'].mean():.2f}%")
print(f"Mejora promedio MSE: {df_sorted['Mejora_MSE'].mean():.2f}%")

## 2. -- 'diabetes.csv' Dataset

### 2.1 -- 20 Minutes Training

In [ ]:
# Cargar dataset
df = pd.read_csv('diabetes.csv')

# CASO DIABETES
X = df.drop('target', axis=1).values
y = df['target'].values

# CASO CALIFORNIA (descomentar si se usa otro dataset)
#X = df.drop('MedHouseVal', axis=1).values
#y = df['MedHouseVal'].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

print(f"\n{'='*70}")
print(f"EVALUACIÓN DEL SISTEMA")
print(f"{'='*70}")
print(f"Dataset: {X.shape[0]} instancias, {X.shape[1]} features")
print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"{'='*70}\n")

# ========================================================================
# BASELINE: Modelo sin optimización
# ========================================================================
print(f"\n{'='*70}")
print(f"BASELINE (Sin Optimización)")
print(f"{'='*70}")

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

baseline = Ridge(alpha=1.0, random_state=42)
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_mse = mean_squared_error(y_test, baseline_preds)

print(f"MAE: {baseline_mae:.4f}")
print(f"MSE: {baseline_mse:.4f}")
print(f"Features utilizadas: {X_train.shape[1]}")

# ========================================================================
# OPTIMIZACIÓN CON PROGRAMACIÓN GENÉTICA + FEATURE SELECTION
# ========================================================================
print(f"\n{'='*70}")
print(f"OPTIMIZACIÓN EVOLUTIVA")
print(f"{'='*70}")

# Crear optimizador
gp_optimizer = EvolutionaryOptimizer(
maxtime=1200,  # 20 minutos (ajusta según necesites)
)

# Entrenar el optimizador (aprende transformaciones)
gp_optimizer.fit(X_train, y_train)

# Transformar los datos (aplicar las transformaciones aprendidas)
X_train_optimized = gp_optimizer.transform(X_train)
X_test_optimized = gp_optimizer.transform(X_test)

if gp_optimizer.feature_selection_ is not None:
    n_selected = np.sum(gp_optimizer.feature_selection_)
    n_total = len(gp_optimizer.feature_selection_)
    print(f"Features seleccionadas: {n_selected}/{n_total}")
    
    # Mostrar cuáles features se seleccionaron
    print(f"\nFeatures seleccionadas:")
    selected_indices = np.where(gp_optimizer.feature_selection_)[0]
    for idx in selected_indices:
        if idx < X.shape[1]:
            print(f"  X{idx} (original)")
        else:
            tree_idx = idx - X.shape[1]
            if tree_idx < len(gp_optimizer.best_trees_):
                print(f"  {gp_optimizer.best_trees_[tree_idx].to_string()} (generada)")

### Performance Evaluation

In [ ]:
# Modelos adicionales para probar
additional_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'SVR': SVR(),
    'XGBoost': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'KernelRidge': KernelRidge(alpha=1.0)
}

print(f"\n{'='*80}")
print("PROBANDO MODELOS ADICIONALES")
print(f"{'='*80}")

results = []

for name, model in additional_models.items():
    try:
        print(f"\nProbando {name}...")
        
        # Baseline
        model_baseline = clone(model)
        model_baseline.fit(X_train, y_train)
        baseline_preds = model_baseline.predict(X_test)
        
        baseline_mae = mean_absolute_error(y_test, baseline_preds)
        baseline_mse = mean_squared_error(y_test, baseline_preds)
        
        # Con optimización
        model_optimized = clone(model)
        model_optimized.fit(X_train_optimized, y_train)
        optimized_preds = model_optimized.predict(X_test_optimized)
        
        optimized_mae = mean_absolute_error(y_test, optimized_preds)
        optimized_mse = mean_squared_error(y_test, optimized_preds)
        
        # Mejoras
        mae_improvement = ((baseline_mae - optimized_mae) / baseline_mae * 100)
        mse_improvement = ((baseline_mse - optimized_mse) / baseline_mse * 100)
        
        results.append({
            'Modelo': name,
            'MAE_Base': baseline_mae,
            'MAE_Opt': optimized_mae,
            'Mejora_MAE': mae_improvement,
            'MSE_Base': baseline_mse,
            'MSE_Opt': optimized_mse,
            'Mejora_MSE': mse_improvement
        })
        
        print(f"  MAE: {baseline_mae:.4f} → {optimized_mae:.4f} ({mae_improvement:+.2f}%)")
        print(f"  MSE: {baseline_mse:.4f} → {optimized_mse:.4f} ({mse_improvement:+.2f}%)")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Mostrar resumen
print(f"\n{'='*100}")
print("RESUMEN COMPLETO - TODOS LOS MODELOS")
print(f"{'='*100}")

df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values('Mejora_MSE', ascending=False)

print(f"{'Modelo':<15} {'MAE Base':<10} {'MAE Opt':<10} {'Mejora MAE':<12} {'MSE Base':<10} {'MSE Opt':<10} {'Mejora MSE':<12}")
print("-" * 100)

for _, row in df_sorted.iterrows():
    print(f"{row['Modelo']:<15} {row['MAE_Base']:<10.4f} {row['MAE_Opt']:<10.4f} "
          f"{row['Mejora_MAE']:+<12.2f}% {row['MSE_Base']:<10.4f} {row['MSE_Opt']:<10.4f} "
          f"{row['Mejora_MSE']:+<12.2f}%")

# Estadísticas finales
print(f"\n{'='*60}")
print("ESTADÍSTICAS GENERALES")
print(f"{'='*60}")
print(f"Modelos que mejoraron MAE: {len(df_sorted[df_sorted['Mejora_MAE'] > 0])}/{len(df_sorted)}")
print(f"Modelos que mejoraron MSE: {len(df_sorted[df_sorted['Mejora_MSE'] > 0])}/{len(df_sorted)}")
print(f"Mejor mejora MAE: {df_sorted['Mejora_MAE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MAE'].idxmax(), 'Modelo']})")
print(f"Mejor mejora MSE: {df_sorted['Mejora_MSE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MSE'].idxmax(), 'Modelo']})")
print(f"Mejora promedio MAE: {df_sorted['Mejora_MAE'].mean():.2f}%")
print(f"Mejora promedio MSE: {df_sorted['Mejora_MSE'].mean():.2f}%")

### 2.2 -- 1 Hour Training

In [ ]:
# Cargar dataset
df = pd.read_csv('diabetes.csv')

# CASO DIABETES
X = df.drop('target', axis=1).values
y = df['target'].values

# CASO CALIFORNIA (descomentar si se usa otro dataset)
#X = df.drop('MedHouseVal', axis=1).values
#y = df['MedHouseVal'].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

print(f"\n{'='*70}")
print(f"EVALUACIÓN DEL SISTEMA")
print(f"{'='*70}")
print(f"Dataset: {X.shape[0]} instancias, {X.shape[1]} features")
print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"{'='*70}\n")

# ========================================================================
# BASELINE: Modelo sin optimización
# ========================================================================
print(f"\n{'='*70}")
print(f"BASELINE (Sin Optimización)")
print(f"{'='*70}")

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

baseline = Ridge(alpha=1.0, random_state=42)
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_mse = mean_squared_error(y_test, baseline_preds)

print(f"MAE: {baseline_mae:.4f}")
print(f"MSE: {baseline_mse:.4f}")
print(f"Features utilizadas: {X_train.shape[1]}")

# ========================================================================
# OPTIMIZACIÓN CON PROGRAMACIÓN GENÉTICA + FEATURE SELECTION
# ========================================================================
print(f"\n{'='*70}")
print(f"OPTIMIZACIÓN EVOLUTIVA")
print(f"{'='*70}")

# Crear optimizador
gp_optimizer = EvolutionaryOptimizer(
maxtime=3600,  # 20 minutos (ajusta según necesites)
)

# Entrenar el optimizador (aprende transformaciones)
gp_optimizer.fit(X_train, y_train)

# Transformar los datos (aplicar las transformaciones aprendidas)
X_train_optimized = gp_optimizer.transform(X_train)
X_test_optimized = gp_optimizer.transform(X_test)

if gp_optimizer.feature_selection_ is not None:
    n_selected = np.sum(gp_optimizer.feature_selection_)
    n_total = len(gp_optimizer.feature_selection_)
    print(f"Features seleccionadas: {n_selected}/{n_total}")
    
    # Mostrar cuáles features se seleccionaron
    print(f"\nFeatures seleccionadas:")
    selected_indices = np.where(gp_optimizer.feature_selection_)[0]
    for idx in selected_indices:
        if idx < X.shape[1]:
            print(f"  X{idx} (original)")
        else:
            tree_idx = idx - X.shape[1]
            if tree_idx < len(gp_optimizer.best_trees_):
                print(f"  {gp_optimizer.best_trees_[tree_idx].to_string()} (generada)")

### Performance Evaluation

In [ ]:
# Modelos adicionales para probar
additional_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'SVR': SVR(),
    'XGBoost': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'KernelRidge': KernelRidge(alpha=1.0)
}

print(f"\n{'='*80}")
print("PROBANDO MODELOS ADICIONALES")
print(f"{'='*80}")

results = []

for name, model in additional_models.items():
    try:
        print(f"\nProbando {name}...")
        
        # Baseline
        model_baseline = clone(model)
        model_baseline.fit(X_train, y_train)
        baseline_preds = model_baseline.predict(X_test)
        
        baseline_mae = mean_absolute_error(y_test, baseline_preds)
        baseline_mse = mean_squared_error(y_test, baseline_preds)
        
        # Con optimización
        model_optimized = clone(model)
        model_optimized.fit(X_train_optimized, y_train)
        optimized_preds = model_optimized.predict(X_test_optimized)
        
        optimized_mae = mean_absolute_error(y_test, optimized_preds)
        optimized_mse = mean_squared_error(y_test, optimized_preds)
        
        # Mejoras
        mae_improvement = ((baseline_mae - optimized_mae) / baseline_mae * 100)
        mse_improvement = ((baseline_mse - optimized_mse) / baseline_mse * 100)
        
        results.append({
            'Modelo': name,
            'MAE_Base': baseline_mae,
            'MAE_Opt': optimized_mae,
            'Mejora_MAE': mae_improvement,
            'MSE_Base': baseline_mse,
            'MSE_Opt': optimized_mse,
            'Mejora_MSE': mse_improvement
        })
        
        print(f"  MAE: {baseline_mae:.4f} → {optimized_mae:.4f} ({mae_improvement:+.2f}%)")
        print(f"  MSE: {baseline_mse:.4f} → {optimized_mse:.4f} ({mse_improvement:+.2f}%)")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Mostrar resumen
print(f"\n{'='*100}")
print("RESUMEN COMPLETO - TODOS LOS MODELOS")
print(f"{'='*100}")

df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values('Mejora_MSE', ascending=False)

print(f"{'Modelo':<15} {'MAE Base':<10} {'MAE Opt':<10} {'Mejora MAE':<12} {'MSE Base':<10} {'MSE Opt':<10} {'Mejora MSE':<12}")
print("-" * 100)

for _, row in df_sorted.iterrows():
    print(f"{row['Modelo']:<15} {row['MAE_Base']:<10.4f} {row['MAE_Opt']:<10.4f} "
          f"{row['Mejora_MAE']:+<12.2f}% {row['MSE_Base']:<10.4f} {row['MSE_Opt']:<10.4f} "
          f"{row['Mejora_MSE']:+<12.2f}%")

# Estadísticas finales
print(f"\n{'='*60}")
print("ESTADÍSTICAS GENERALES")
print(f"{'='*60}")
print(f"Modelos que mejoraron MAE: {len(df_sorted[df_sorted['Mejora_MAE'] > 0])}/{len(df_sorted)}")
print(f"Modelos que mejoraron MSE: {len(df_sorted[df_sorted['Mejora_MSE'] > 0])}/{len(df_sorted)}")
print(f"Mejor mejora MAE: {df_sorted['Mejora_MAE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MAE'].idxmax(), 'Modelo']})")
print(f"Mejor mejora MSE: {df_sorted['Mejora_MSE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MSE'].idxmax(), 'Modelo']})")
print(f"Mejora promedio MAE: {df_sorted['Mejora_MAE'].mean():.2f}%")
print(f"Mejora promedio MSE: {df_sorted['Mejora_MSE'].mean():.2f}%")

## 3. -- 'diabetes_smote.csv'

In [ ]:
# Cargar dataset
df = pd.read_csv('diabetes_smote.csv')

# CASO DIABETES
X = df.drop('target', axis=1).values
y = df['target'].values

# CASO CALIFORNIA (descomentar si se usa otro dataset)
#X = df.drop('MedHouseVal', axis=1).values
#y = df['MedHouseVal'].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

print(f"\n{'='*70}")
print(f"EVALUACIÓN DEL SISTEMA")
print(f"{'='*70}")
print(f"Dataset: {X.shape[0]} instancias, {X.shape[1]} features")
print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"{'='*70}\n")

# ========================================================================
# BASELINE: Modelo sin optimización
# ========================================================================
print(f"\n{'='*70}")
print(f"BASELINE (Sin Optimización)")
print(f"{'='*70}")

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

baseline = Ridge(alpha=1.0, random_state=42)
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_mse = mean_squared_error(y_test, baseline_preds)

print(f"MAE: {baseline_mae:.4f}")
print(f"MSE: {baseline_mse:.4f}")
print(f"Features utilizadas: {X_train.shape[1]}")

# ========================================================================
# OPTIMIZACIÓN CON PROGRAMACIÓN GENÉTICA + FEATURE SELECTION
# ========================================================================
print(f"\n{'='*70}")
print(f"OPTIMIZACIÓN EVOLUTIVA")
print(f"{'='*70}")

# Crear optimizador
gp_optimizer = EvolutionaryOptimizer(
maxtime=3600,  # 20 minutos (ajusta según necesites)
)

# Entrenar el optimizador (aprende transformaciones)
gp_optimizer.fit(X_train, y_train)

# Transformar los datos (aplicar las transformaciones aprendidas)
X_train_optimized = gp_optimizer.transform(X_train)
X_test_optimized = gp_optimizer.transform(X_test)

if gp_optimizer.feature_selection_ is not None:
    n_selected = np.sum(gp_optimizer.feature_selection_)
    n_total = len(gp_optimizer.feature_selection_)
    print(f"Features seleccionadas: {n_selected}/{n_total}")
    
    # Mostrar cuáles features se seleccionaron
    print(f"\nFeatures seleccionadas:")
    selected_indices = np.where(gp_optimizer.feature_selection_)[0]
    for idx in selected_indices:
        if idx < X.shape[1]:
            print(f"  X{idx} (original)")
        else:
            tree_idx = idx - X.shape[1]
            if tree_idx < len(gp_optimizer.best_trees_):
                print(f"  {gp_optimizer.best_trees_[tree_idx].to_string()} (generada)")

### Performance Evaluation

In [ ]:
# Modelos adicionales para probar
additional_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'SVR': SVR(),
    'XGBoost': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'KernelRidge': KernelRidge(alpha=1.0)
}

print(f"\n{'='*80}")
print("PROBANDO MODELOS ADICIONALES")
print(f"{'='*80}")

results = []

for name, model in additional_models.items():
    try:
        print(f"\nProbando {name}...")
        
        # Baseline
        model_baseline = clone(model)
        model_baseline.fit(X_train, y_train)
        baseline_preds = model_baseline.predict(X_test)
        
        baseline_mae = mean_absolute_error(y_test, baseline_preds)
        baseline_mse = mean_squared_error(y_test, baseline_preds)
        
        # Con optimización
        model_optimized = clone(model)
        model_optimized.fit(X_train_optimized, y_train)
        optimized_preds = model_optimized.predict(X_test_optimized)
        
        optimized_mae = mean_absolute_error(y_test, optimized_preds)
        optimized_mse = mean_squared_error(y_test, optimized_preds)
        
        # Mejoras
        mae_improvement = ((baseline_mae - optimized_mae) / baseline_mae * 100)
        mse_improvement = ((baseline_mse - optimized_mse) / baseline_mse * 100)
        
        results.append({
            'Modelo': name,
            'MAE_Base': baseline_mae,
            'MAE_Opt': optimized_mae,
            'Mejora_MAE': mae_improvement,
            'MSE_Base': baseline_mse,
            'MSE_Opt': optimized_mse,
            'Mejora_MSE': mse_improvement
        })
        
        print(f"  MAE: {baseline_mae:.4f} → {optimized_mae:.4f} ({mae_improvement:+.2f}%)")
        print(f"  MSE: {baseline_mse:.4f} → {optimized_mse:.4f} ({mse_improvement:+.2f}%)")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Mostrar resumen
print(f"\n{'='*100}")
print("RESUMEN COMPLETO - TODOS LOS MODELOS")
print(f"{'='*100}")

df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values('Mejora_MSE', ascending=False)

print(f"{'Modelo':<15} {'MAE Base':<10} {'MAE Opt':<10} {'Mejora MAE':<12} {'MSE Base':<10} {'MSE Opt':<10} {'Mejora MSE':<12}")
print("-" * 100)

for _, row in df_sorted.iterrows():
    print(f"{row['Modelo']:<15} {row['MAE_Base']:<10.4f} {row['MAE_Opt']:<10.4f} "
          f"{row['Mejora_MAE']:+<12.2f}% {row['MSE_Base']:<10.4f} {row['MSE_Opt']:<10.4f} "
          f"{row['Mejora_MSE']:+<12.2f}%")

# Estadísticas finales
print(f"\n{'='*60}")
print("ESTADÍSTICAS GENERALES")
print(f"{'='*60}")
print(f"Modelos que mejoraron MAE: {len(df_sorted[df_sorted['Mejora_MAE'] > 0])}/{len(df_sorted)}")
print(f"Modelos que mejoraron MSE: {len(df_sorted[df_sorted['Mejora_MSE'] > 0])}/{len(df_sorted)}")
print(f"Mejor mejora MAE: {df_sorted['Mejora_MAE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MAE'].idxmax(), 'Modelo']})")
print(f"Mejor mejora MSE: {df_sorted['Mejora_MSE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MSE'].idxmax(), 'Modelo']})")
print(f"Mejora promedio MAE: {df_sorted['Mejora_MAE'].mean():.2f}%")
print(f"Mejora promedio MSE: {df_sorted['Mejora_MSE'].mean():.2f}%")